# The Fast Fourier Transform

This notebook explores the ideas behind the Fast Fourier Transform or FFT as a method of _quickly_ calculating the DFT of a sample vector.

## Setup

We first do some imports of common libraries. While libraries like NumPy already contain a way more efficient [FFT](https://numpy.org/doc/stable/reference/generated/numpy.fft.fft.html) implementation, the purpose of this notebook is to write our own (very limit) version.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Little Helpers

Small helper functions to check if $n$ is a power of $2$.

In [ ]:
def ispow2(n: int):
    return (n & (n - 1) == 0) and n != 0

## Implementing the FFT

Here we finally implement our very simple variant of the FFT. We only handle inputs of length $2^k, k \in \mathbb{N}_{0}$. Other implementations of FFT exist, in which the input can be of any length (such as NumPy's [`numpy.fft.fft()`](https://numpy.org/doc/stable/reference/generated/numpy.fft.fft.html)), but those are a little more complicated.

In [ ]:
def fft(samples: np.ndarray):
    n = samples.shape[0]
    # assure order of samples is power of 2
    assert ispow2(n)

    # ensure recurson stops!
    if n == 1:
        return samples

    unity_root = np.exp(-2j * np.pi / n)

    # select every other element (0, 2, 4, ...)
    samples_even = samples[::2]
    # select every other element ... starting from 1 (1, 3, 5, ...)
    samples_odd = samples[1::2]

    y_even, y_odd = fft(samples_even), fft(samples_odd)

    y = np.empty(n, dtype=complex)
    for k in range(n // 2):
        y[k] = y_even[k] + y_odd[k] * unity_root ** k
        y[k + n // 2] = y_even[k] - y_odd[k] * unity_root ** k

    return y

## Testing

Play through some values and check against a _trusted_ FFT implementation.

In [ ]:
samples = np.arange(2 ** 3)

# Let's test our FFT
a = fft(samples)

# For comparison reasons, let's also try NumPy's FFT
b = np.fft.fft(samples)

print(f"Our FFT:\n{a}\n")
print(f"NumPy FFT:\n{b}\n")
print(f"Are they the same?\n{np.allclose(a, b)}\n")

## Understanding the Output of the FFT

Let's try a bit more realitic example of a square wave.

In [ ]:
s = np.pad(np.ones(102), pad_width=(0, 922))

plt.figure(figsize=(15, 3))
plt.title(f"Square wave with {s.shape[0]} samples: 1/10th duty cycle")
plt.xlabel("sample")
plt.ylabel("amplitude")
plt.plot(s)
plt.grid(True)


plt.show()

Calculating the FFT and naively plotting the output vector reveals something seemingly odd about the order of the returned coefficients.

The frequency components returned by the FFT seem to be in the _wrong_ order. Well, not necessarily wrong, but maybe unintuitive at first. To understand what's going on, think of the indicies we use to the frequency component vector as angles around the **complex unit-circle**. This circle is cut into N equal slices, as we have N elements in our component vector. We start index $0$, which represents an angle of $0^{\circ}$ or $0 \cdot \pi$ radians. This is the $0$ Hz, or DC offset of our signal. In our case, $1 / 10$ times the amplitude of our rectangular pulse. Next we walk around the unit-circle counter-clockwise. Index $n = 1$ would be $\frac{1}{N} \cdot 2 \pi$. The same logic applies to every other index. The general formula to convert from index $n$ to frequency component $\omega$ is: $$\omega = \frac{n}{N} \cdot 2 \pi \cdot \omega_{0}$$ With $\omega_{0}$ representing the sampling frequency.

In [ ]:
S = fft(s)
S_abs = np.abs(S) / S.shape[0]

plt.figure(figsize=(15, 3))
plt.title("FFT output")
plt.xlabel("frequency bins")
plt.ylabel("absolute")
plt.plot(S_abs)
plt.grid(True)

plt.annotate("bin 0: $0 \\cdot \\omega_{{0}}$ or DC offset",
             xy=(0, S_abs[0]),
             xytext=(-50, 20),
             xycoords="data",
             textcoords="offset pixels",
             arrowprops={'arrowstyle': 'wedge'})
plt.annotate(f"bin {1}: $\\frac{{1}}{{{S_abs.shape[0]}}} "
             "\\cdot 2 \\pi \\cdot \\omega_{{0}}$",
             xy=(1, S_abs[1]),
             xytext=(100, -20),
             xycoords="data",
             textcoords="offset pixels",
             arrowprops={'arrowstyle': 'wedge'})
plt.annotate(f"bin {100}: $\\frac{{100}}{{{S_abs.shape[0]}}} "
             "\\cdot 2 \\pi \\cdot \\omega_{{0}}$",
             xy=(100, S_abs[100]),
             xytext=(50, 20),
             xycoords="data",
             textcoords="offset pixels",
             arrowprops={'arrowstyle': 'wedge'})
plt.annotate(f"bin {S_abs.shape[0] // 2}: $\\omega_{{0}} "
             "\\cdot \\pi$ or Nyquist Frequency",
             xy=(S_abs.shape[0] // 2, S_abs[S_abs.shape[0] // 2]),
             xytext=(-100, 20),
             xycoords="data",
             textcoords="offset pixels",
             arrowprops={'arrowstyle': 'wedge'})
plt.annotate(f"bin {950}: $\\frac{{950}}{{{S_abs.shape[0]}}} "
             "\\cdot 2 \\pi \\cdot \\omega_{{0}}$",
             xy=(950, S_abs[950]),
             xytext=(-100, 40),
             xycoords="data",
             textcoords="offset pixels",
             arrowprops={'arrowstyle': 'wedge'})
plt.annotate(f"bin {S_abs.shape[0] - 1}: "
             f"$\\frac{{{S_abs.shape[0] - 1}}}{{{S_abs.shape[0]}}} "
             "\\cdot 2 \\pi \\cdot \\omega_{{0}}$",
             xy=(S_abs.shape[0] - 1, S_abs[-1]),
             xytext=(-100, 20),
             xycoords="data",
             textcoords="offset pixels",
             arrowprops={'arrowstyle': 'wedge'})

plt.show()

To fix this, we can simply reshuffle the frequency bins a bit. NumPy conveniently offers the function [`np.fft.fftshift()`](https://numpy.org/devdocs/reference/generated/numpy.fft.fftshift.html?highlight=fftshift#numpy.fft.fftshift), which shifts the zero-frequency component to the center of the array. Around that, the left half of the array is filled with the frequency components $\pi \le \omega \lt 2 \pi$ (_negative_ frequencies) and the right half with components $0 \lt \omega \lt \pi$. **Neat!**

In [ ]:
S_pwr = S_abs / S.shape[0]

plt.figure(figsize=(15, 3))
plt.title("Frequency Spectrum of square wave")
plt.xlabel("frequency bins")
plt.ylabel("power")
plt.plot(np.arange(-S.shape[0] // 2, S.shape[0] // 2), np.fft.fftshift(S_pwr))
plt.grid(True)

plt.annotate("bin 0: $0 \\cdot \\omega_{{0}}$ or DC offset",
             xy=(0, S_pwr[0]),
             xytext=(50, -10),
             xycoords="data",
             textcoords="offset pixels",
             arrowprops={'arrowstyle': 'wedge'})

plt.show()